In [ ]:
!pip install -r requirements.txt

In [2]:
import os
import zipfile
import pandas as pd

In [3]:
# with zipfile.ZipFile('archive.zip', 'r') as zip_ref:
#     zip_ref.extractall('data')

In [4]:
df = pd.read_csv(
    'data/jd.csv',
#     nrows=100
)

In [5]:
import nltk
nltk.download('punkt')
from nltk import sent_tokenize

[nltk_data] Downloading package punkt to /home/ec2-user/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [6]:
sents = []
for jd in df.FullDescription:
    sents.extend(sent_tokenize(jd))

In [21]:
sents = pd.DataFrame(data={'sents': sents})

In [22]:
sents['length'] = sents.sents.map(len)

In [28]:
np.percentile(lens, 95)

426.0

In [86]:
from tokenizers import Tokenizer
from tokenizers.models import BPE, WordPiece
from tokenizers.trainers import BpeTrainer, WordPieceTrainer
from tokenizers import pre_tokenizers
from tokenizers.pre_tokenizers import Digits, Whitespace, ByteLevel, BertPreTokenizer
from tokenizers import normalizers
from tokenizers import processors
from tokenizers.normalizers import NFD, Lowercase, StripAccents, BertNormalizer

# bert
pre_tokenizer = pre_tokenizers.Sequence([BertPreTokenizer(), Digits(individual_digits=True)])
normalizer = normalizers.Sequence([BertNormalizer()])
tokenizer = Tokenizer(WordPiece(unk_token="[UNK]"))
tokenizer.pre_tokenizer = pre_tokenizer
tokenizer.normalizer = normalizer
tokenizer.processors = processors.BertProcessing(('[CLS]', 2), ('[SEP]', 3))
trainer = WordPieceTrainer(special_tokens=["[PAD]", "[UNK]", "[CLS]", "[SEP]", "[MASK]"], vocab_size=36000)


# bpe
# pre_tokenizer = pre_tokenizers.Sequence([ByteLevel(), Digits(individual_digits=True)])
# normalizer = normalizers.Sequence([Lowercase(), StripAccents(), NFD()])
# tokenizer = Tokenizer(BPE(unk_token="[UNK]"))
# tokenizer.pre_tokenizer = pre_tokenizer
# tokenizer.normalizer = normalizer
# tokenizer.processors = processors.ByteLevel()
# trainer = BpeTrainer(special_tokens=["[PAD]", "[UNK]", "[CLS]", "[SEP]", "[MASK]"], vocab_size=36000)


In [87]:
tokenizer.train_from_iterator(sents[sents['length'] < 500]['sents'], trainer=trainer)

In [96]:
enc = tokenizer.encode('(thisvector)', add_special_tokens=False)
enc.ids, enc.tokens

([10, 220, 26524, 7192, 11], ['(', 'this', '##vec', '##tor', ')'])

In [92]:
tokenizer.decode(enc.ids, skip_special_tokens=True).replace(' ##', "")

'( thisvector )'

In [93]:
tokenizer.save('data/jd_tokenizer_wordpiece.json')

In [ ]:
!wget https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt -o data/bert-base-uncased-vocab.txt